# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (linear problem)

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D.

We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **weak imposition of Dirichlet BCs**: this requires an introduction of a multiplier $\lambda$ which is restricted to $\Gamma$, and solves
$$
\text{find } w, \lambda \in V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega \nabla w \cdot \nabla v + \int_\Gamma \lambda v = \int_\Omega f v, & \forall v \in V,\\
\int_\Gamma w \mu = \int_\Gamma g \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

This example is a prototypical case of problems containing subdomain/boundary restricted variables (the Lagrange multiplier, in this case).

In [ ]:
import numpy as np
from petsc4py import PETSc
from ufl import grad, inner, Measure, TestFunction, TrialFunction
from dolfinx import DirichletBC, Function, FunctionSpace, MPI, solve
from dolfinx.cpp.la import create_petsc_index_sets, GhostBlockLayout, VecSubVectorReadWrapper
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                         create_vector_block, DofMapRestriction, locate_dofs_topological)
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Mesh

In [ ]:
with XDMFFile(MPI.comm_world, "data/circle.xdmf") as infile:
    mesh = infile.read_mesh(GhostMode.none)
with XDMFFile(MPI.comm_world, "data/circle_subdomains.xdmf") as infile:
    subdomains = infile.read_mf_size_t(mesh)
with XDMFFile(MPI.comm_world, "data/circle_boundaries.xdmf") as infile:
    boundaries = infile.read_mf_size_t(mesh)
facets_Gamma = np.where(boundaries.values == 1)[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Weak imposition of Dirichlet BCs

In [ ]:
# Define a function space
V = FunctionSpace(mesh, ("Lagrange", 2))

In [ ]:
# Define restrictions
dofs_V = np.arange(0, V.dofmap.index_map.block_size * (
    V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts))
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
restriction_V = DofMapRestriction(V.dofmap, dofs_V)
restriction_V_Gamma = DofMapRestriction(V.dofmap, dofs_V_Gamma)
restriction = [restriction_V, restriction_V_Gamma]

In [ ]:
# Define trial and test functions
(u, l) = (TrialFunction(V), TrialFunction(V))
(v, m) = (TestFunction(V), TestFunction(V))

In [ ]:
# Define problem block forms
g = Function(V)
g.interpolate(lambda x: np.sin(3 * x[0] + 1) * np.sin(3 * x[1] + 1))
a = [[inner(grad(u), grad(v)) * dx, l * v * ds],
     [u * m * ds, None]]
f = [v * dx, g * m * ds]

In [ ]:
# Assemble the block linear system
A = assemble_matrix_block(a, bcs=[], restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=[], restriction=restriction)

In [ ]:
# Solve
ul = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, ul)

In [ ]:
# Split the block solution in components
(u, l) = (Function(V), Function(V))
index_sets = create_petsc_index_sets(
    [V.dofmap.index_map] * 2,
    ghost_block_layout=GhostBlockLayout.trailing)
restricted_index_sets = create_petsc_index_sets(
    [restriction_.index_map for restriction_ in restriction],
    ghost_block_layout=GhostBlockLayout.trailing)
unrestricted_to_restricted = [restriction_.unrestricted_to_restricted for restriction_ in restriction]
for i, sub in enumerate((u, l)):
    with sub.vector.localForm() as sub_local:
        sub_local[:] = VecSubVectorReadWrapper(ul, index_sets[i], restricted_index_sets[i],
                                               unrestricted_to_restricted[i]).content
    sub.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
plot(u)

In [ ]:
plot(l)

### Strong imposition of Dirichlet BCs

In [ ]:
# Define Dirichlet BC object on Gamma
bc_ex = DirichletBC(g, dofs_V_Gamma)

In [ ]:
# Solve
u_ex = Function(V)
solve(a[0][0] == f[0], u_ex, bc_ex,
      petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
plot(u_ex)

### Comparison and error compuation

In [ ]:
u_ex_norm = np.sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u_ex), grad(u_ex)) * dx)))
err_norm = np.sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u_ex - u), grad(u_ex - u)) * dx)))
print("Relative error is equal to", err_norm / u_ex_norm)
assert np.isclose(err_norm / u_ex_norm, 0., atol=1.e-10)